In [1]:
from os import environ

# environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HF_HOME"] = "/mnt/workspaces/hf_home"
environ["HTTP_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["HTTPS_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"


from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	add_myscale_retrieval
	mcma_prompt
	mcsa_prompt
	mcsa_fewshot_mmlu
Models:
	remote-llm
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


In [2]:
import yaml
from retrieval_qa_benchmark.utils.config import load
config = load(open("../config/test.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
# print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 5085.72it/s]


In [4]:
data = evaluator.transform(evaluator.dataset[0])
# print(evaluator.transform(evaluator.dataset[0]).raw_question)
# evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]
data

QARecord(id='4e8db4a928333356588d2b89477e5d5bcb95532cd8afa355456ac94e13fd1849', question='Unlike most other early civilizations, Minoan culture shows little evidence of:', answer='conspicuous consumption by elites.', type='MCSA', context=None, choices=['trade.', 'warfare.', 'the development of a common religion.', 'conspicuous consumption by elites.'], stack=[])

In [7]:
print(evaluator.llm.convert_record(data))

The following are multiple choice questions (with answers):

Question: Unlike most other early civilizations, Minoan culture shows little evidence of:
A. trade.	B. warfare.	C. the development of a common religion.	D. conspicuous consumption by elites.
Answer: 


In [5]:
print(str(PROFILER))

	======== Profile Result ========
BaseTransform.chain                               	Count:     324	AvgTime:    0.05 ms
BaseDataset.__getitem__                           	Count:       2	AvgTime:    0.02 ms
TransformChain.__call__                           	Count:       2	AvgTime:    0.01 ms



In [ ]:
matched = evaluator()

In [ ]:
print(str(PROFILER))

In [ ]:
with open("results.with-retrieval.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction

mismatched = [pred for pred in matched[1] if pred.matched]

In [ ]:
with open("results.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in mismatched]))